In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from tqdm import tqdm
import pandas as pd
import ast
import os

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, output_size=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, lengths):
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed_input)
        unpacked_output, _ = pad_packed_sequence(packed_output, batch_first=True)
        out = self.fc(hn[-1])
        return out

In [3]:
cityA = pd.read_csv("../../Datasets/Task 2/frequent_sequences_A.csv")
cityA = list(cityA['Pattern'])
cityA_sequence = []

for str_seq in cityA:
    tuple_seq = ast.literal_eval(str_seq)
    cityA_sequence.append(tuple_seq)

train_sequencesA, test_sequencesA = train_test_split(cityA_sequence, test_size=0.2, random_state=42)

In [4]:
cityB = pd.read_csv("../../Datasets/Task 2/frequent_sequences_B.csv")
cityB = list(cityB['Pattern'])
cityB_sequence = []

for str_seq in cityB:
    tuple_seq = ast.literal_eval(str_seq)
    cityB_sequence.append(tuple_seq)

train_sequencesB, test_sequencesB = train_test_split(cityB_sequence, test_size=0.2, random_state=42)

In [5]:
cityC = pd.read_csv("../../Datasets/Task 2/frequent_sequences_C.csv")
cityC = list(cityC['Pattern'])
cityC_sequence = []

for str_seq in cityC:
    tuple_seq = ast.literal_eval(str_seq)
    cityC_sequence.append(tuple_seq)

train_sequencesC, test_sequencesC = train_test_split(cityC_sequence, test_size=0.2, random_state=42)

In [6]:
cityD = pd.read_csv("../../Datasets/Task 2/frequent_sequences_D.csv")
cityD = list(cityD['Pattern'])
cityD_sequence = []

for str_seq in cityD:
    tuple_seq = ast.literal_eval(str_seq)
    cityD_sequence.append(tuple_seq)

train_sequencesD, test_sequencesD = train_test_split(cityD_sequence, test_size=0.2, random_state=42)

In [7]:
def get_inputs_labels(dataset):
    inputs = []
    labels = []
    for seq in dataset:
        inputs.append(torch.tensor(seq[:-1], dtype=torch.float32))
        labels.append(torch.tensor(seq[-1], dtype=torch.float32))

    return inputs, labels

In [ ]:
def train_model(train_sequences, weights_path):
    X_train, y_train = get_inputs_labels(train_sequences)

    weights_path = os.path.join("weights", weights_path)


    model = LSTMModel()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 100

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0

        for seq, target in tqdm(zip(X_train, y_train), total=len(X_train), desc=f"Epoch {epoch + 1}/{num_epochs}"):       
            seq_length = torch.tensor([len(seq)])  
            padded_seq = pad_sequence([seq], batch_first=True, padding_value=0).to(torch.float32)

            target = target.unsqueeze(0)  
            optimizer.zero_grad()

            output = model(padded_seq, seq_length)

            loss = criterion(output, target)
            epoch_loss += loss.item()

            loss.backward()
            optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / len(X_train):.4f}")
            
    print("Training complete.")

    torch.save(model.state_dict(), weights_path)
    print('saved to: ', weights_path)

In [16]:
def test_model(test_sequences, weights_path):
    X_test, y_test = get_inputs_labels(test_sequences)

    weights_path = os.path.join("weights", weights_path)


    model = LSTMModel()
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    
    predictions = []

    with torch.no_grad():
        for seq in X_test:
            # Prepare sequence
            seq_length = torch.tensor([len(seq)])  # Sequence length
            padded_seq = pad_sequence([seq], batch_first=True).float()  # Convert to float if needed

            # Forward pass to get the prediction
            pred = model(padded_seq, seq_length)
            predictions.append(pred.squeeze().cpu())  # Append the prediction, removing extra dimensions

    # Display predictions and compare to true targets
    for i, (pred, true_target) in enumerate(zip(predictions, y_test)):
        print(f"Test Sequence {i+1}:")
        print(f"  Predicted next coordinate: {pred.numpy()}")
        print(f"  True next coordinate: {true_target.numpy()}")

In [ ]:
train_model(train_sequencesA, weights_path="cityA.pth")


Epoch 10/100: 100%|██████████| 17815/17815 [00:25<00:00, 692.00it/s]


Epoch [10/100], Loss: 28.5295


Epoch 20/100: 100%|██████████| 17815/17815 [00:28<00:00, 634.92it/s]


Epoch [20/100], Loss: 24.7511


Epoch 30/100: 100%|██████████| 17815/17815 [00:26<00:00, 672.24it/s]


...

Epoch 90/100: 100%|██████████| 17815/17815 [00:27<00:00, 657.83it/s]


Epoch [90/100], Loss: 23.7092


Epoch 100/100: 100%|██████████| 17815/17815 [00:27<00:00, 658.23it/s]


Epoch [100/100], Loss: 23.5838
Training complete.
saved to:  weights\cityA.pth


FileNotFoundError: [Errno 2] No such file or directory: 'cityA.pth'

In [14]:
test_model(test_sequencesA, weights_path="./weights/cityA.pth")

Test Sequence 1:
  Predicted next coordinate: [176.1075  45.214 ]
  True next coordinate: [176.  48.]
Test Sequence 2:
  Predicted next coordinate: [ 54.77323 168.20311]
  True next coordinate: [ 56. 168.]
Test Sequence 3:
  Predicted next coordinate: [ 18.27745 109.11273]
  True next coordinate: [ 24. 111.]
Test Sequence 4:
  Predicted next coordinate: [ 96.6445 110.6371]
  True next coordinate: [ 97. 111.]
Test Sequence 5:
  Predicted next coordinate: [ 1.5959778 34.46978  ]
  True next coordinate: [ 1. 33.]
Test Sequence 6:
  Predicted next coordinate: [117.64739  96.80162]
  True next coordinate: [117. 101.]
Test Sequence 7:
  Predicted next coordinate: [182.91556 103.50127]
  True next coordinate: [183. 104.]
Test Sequence 8:
  Predicted next coordinate: [143.89758   88.020996]
  True next coordinate: [142.  90.]
Test Sequence 9:
  Predicted next coordinate: [157.70715   37.794777]
  True next coordinate: [156.  40.]
Test Sequence 10:
  Predicted next coordinate: [ 76.88026 100.50

In [17]:
train_model(train_sequencesB, weights_path="cityB.pth")
test_model(test_sequencesB, weights_path="cityB.pth")

Epoch 10/100: 100%|██████████| 11278/11278 [00:17<00:00, 637.02it/s]


Epoch [10/100], Loss: 38.4398


Epoch 20/100: 100%|██████████| 11278/11278 [00:18<00:00, 603.71it/s]


Epoch [20/100], Loss: 33.3492


Epoch 30/100: 100%|██████████| 11278/11278 [00:21<00:00, 534.15it/s]


...

Epoch [80/100], Loss: 28.7028


Epoch 90/100: 100%|██████████| 11278/11278 [00:18<00:00, 618.56it/s]


Epoch [90/100], Loss: 27.9861


Epoch 100/100: 100%|██████████| 11278/11278 [00:18<00:00, 614.99it/s]


Epoch [100/100], Loss: 27.4156
Training complete.
saved to:  weights\cityB.pth
Test Sequence 1:
  Predicted next coordinate: [58.616558 58.03947 ]
  True next coordinate: [60. 58.]
Test Sequence 2:
  Predicted next coordinate: [75.42988  73.705666]
  True next coordinate: [79. 73.]
Test Sequence 3:
  Predicted next coordinate: [72.22191  97.179375]
  True next coordinate: [71. 94.]
Test Sequence 4:
  Predicted next coordinate: [ 93.034    101.275406]
  True next coordinate: [95. 98.]
Test Sequence 5:
  Predicted next coordinate: [19.130383 13.266557]
  True next coordinate: [17. 10.]
Test Sequence 6:
  Predicted next coordinate: [ 56.28024 148.28452]
  True next coordinate: [ 57. 150.]
Test Sequence 7:
  Predicted next coordinate: [35.095963 22.159744]
  True next coordinate: [35. 23.]
Test Sequence 8:
  Predicted next coordinate: [ 69.49091  104.234634]
  True next coordinate: [ 67. 102.]
Test Sequence 9:
  Predicted next coordinate: [49.563587 45.49051 ]
  True next coordinate: [52. 

In [19]:
train_model(train_sequencesC, weights_path="cityC.pth")
test_model(test_sequencesC, weights_path="cityC.pth")

Epoch 10/100: 100%|██████████| 8536/8536 [00:12<00:00, 685.82it/s]


Epoch [10/100], Loss: 28.5608


Epoch 20/100: 100%|██████████| 8536/8536 [00:13<00:00, 644.89it/s]


Epoch [20/100], Loss: 22.1595


Epoch 30/100: 100%|██████████| 8536/8536 [00:12<00:00, 657.56it/s]


...

Epoch [80/100], Loss: 18.8357


Epoch 90/100: 100%|██████████| 8536/8536 [00:13<00:00, 624.34it/s]


Epoch [90/100], Loss: 18.6060


Epoch 100/100: 100%|██████████| 8536/8536 [00:14<00:00, 597.34it/s]


Epoch [100/100], Loss: 18.3399
Training complete.
saved to:  weights\cityC.pth
Test Sequence 1:
  Predicted next coordinate: [ 36.18421  122.793686]
  True next coordinate: [ 38. 123.]
Test Sequence 2:
  Predicted next coordinate: [ 34.29823 159.4317 ]
  True next coordinate: [ 33. 166.]
Test Sequence 3:
  Predicted next coordinate: [ 47.016937 139.72684 ]
  True next coordinate: [ 48. 143.]
Test Sequence 4:
  Predicted next coordinate: [ 31.610598 146.09067 ]
  True next coordinate: [ 25. 168.]
Test Sequence 5:
  Predicted next coordinate: [ 48.393677 145.0056  ]
  True next coordinate: [ 50. 149.]
Test Sequence 6:
  Predicted next coordinate: [ 12.300886 162.01715 ]
  True next coordinate: [ 12. 167.]
Test Sequence 7:
  Predicted next coordinate: [ 54.567833 175.11104 ]
  True next coordinate: [ 58. 180.]
Test Sequence 8:
  Predicted next coordinate: [ 32.95854 152.93703]
  True next coordinate: [ 33. 156.]
Test Sequence 9:
  Predicted next coordinate: [ 32.802402 134.77089 ]
  True 

In [20]:
train_model(train_sequencesD, weights_path="cityD.pth")
test_model(test_sequencesD, weights_path="cityD.pth")

Epoch 10/100: 100%|██████████| 5272/5272 [00:07<00:00, 719.71it/s]


Epoch [10/100], Loss: 30.5797


Epoch 20/100: 100%|██████████| 5272/5272 [00:07<00:00, 704.78it/s]


Epoch [20/100], Loss: 22.2442


Epoch 30/100: 100%|██████████| 5272/5272 [00:07<00:00, 690.86it/s]


...

Epoch [80/100], Loss: 13.8076


Epoch 90/100: 100%|██████████| 5272/5272 [00:08<00:00, 646.97it/s]


Epoch [90/100], Loss: 14.0694


Epoch 100/100: 100%|██████████| 5272/5272 [00:08<00:00, 648.85it/s]


Epoch [100/100], Loss: 13.4797
Training complete.
saved to:  weights\cityD.pth
Test Sequence 1:
  Predicted next coordinate: [142.44157  54.47293]
  True next coordinate: [140.  56.]
Test Sequence 2:
  Predicted next coordinate: [137.7327   85.60194]
  True next coordinate: [139.  81.]
Test Sequence 3:
  Predicted next coordinate: [113.75861 115.61081]
  True next coordinate: [113. 120.]
Test Sequence 4:
  Predicted next coordinate: [185.9648    38.179462]
  True next coordinate: [185.  37.]
Test Sequence 5:
  Predicted next coordinate: [124.76824   61.489464]
  True next coordinate: [124.  62.]
Test Sequence 6:
  Predicted next coordinate: [183.82426     4.0702744]
  True next coordinate: [181.   1.]
Test Sequence 7:
  Predicted next coordinate: [ 86.18219 160.95847]
  True next coordinate: [ 86. 161.]
Test Sequence 8:
  Predicted next coordinate: [146.4375  117.56857]
  True next coordinate: [145. 115.]
Test Sequence 9:
  Predicted next coordinate: [157.35179 170.2582 ]
  True next c